In [ ]:
import numpy as np
from scipy.io import readsav
import scipy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
def _load_data(filename):
    dat = readsav(filename)
    emission = dat['emission_structure']
    return emission[0]

def _find_index(arr,val):
    return np.argmin(abs(arr-val))

In [ ]:
def norm(data):
    mn = data.mean()
    std = data.std()
    return((data-mn)/std)

def rescale(data):
    return (data-data.min())/(data.max()-data.min())

def quantfilt(src,thr=0.9):
    filt = np.quantile(src,thr,axis=0)
    out = np.where(src<filt,0,src)
    return out

# gaussian filtering
def gaussblr(src,filt=(31, 3)):
    src = (rescale(src)*255).astype('uint8')
    out = cv2.GaussianBlur(src,filt,0)
    return rescale(out)

# mean filtering
def meansub(src):
    mn = np.mean(src,axis=1)[:,np.newaxis]
    out = np.absolute(src - mn)
    return rescale(out)

# morphological filtering
def morph(src):
    src = (rescale(src)*255).astype('uint8')
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    mask = cv2.morphologyEx(src, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    return rescale(mask)

In [ ]:
filepath = Path('tv_images')
filename = 'emission_structure_pu_cam240perp_189101.sav'
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = _load_data(filepath / filename)

In [ ]:

fid = 50
tid = _find_index(vid_times,times[fid]) #find frame id for camera image with t=times[fid]
find_x = inverted[fid]
img = inverted[fid].copy()
gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
useHarrisDetector = False # False uses Shi-Tomasi Corner Detector
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x = radii[fid][corners[:,0,0]]
y = elevation[fid][corners[:,0,1]]
print(np.column_stack((x,y)))
# plt.imshow(img)
plt.pcolormesh(radii[fid],elevation[fid],img,shading='auto', cmap='gray')
plt.scatter(x,y,color='red')
plt.title(f"Inverted with Radiation Points, Frame {fid}")
plt.show()

In [ ]:
def convert_center(radii, elevation, value):
    rad_idx = (np.abs(radii - value[0])).argmin()
    elev_idx = (np.abs(elevation - value[1])).argmin()
    return rad_idx, elev_idx

def get_avg_value(array):
    return np.sqrt(np.mean(np.square(array)))

def get_dist_line(point_1, point_2, test_point):
    top = np.abs((point_2[0]-point_1[0])*(point_2[1]-test_point[1])-(point_1[0]-test_point[0])*(point_2[1]-point_1[1]))
    bottom = np.sqrt((point_2[0]-point_1[0])**2+(point_2[1]-point_1[1])**2)
    return top / bottom

In [ ]:
def get_corners(img):
    
    gray = (255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
    corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=False))
    x = corners[:,0,0]
    y = corners[:,0,1]
    
    return np.column_stack((x,y))

def get_corner_values(img, corners, dist, im_size):
    
    bounds = get_bounds(corners, dist, im_size)
    
    avg_values = []
    
    for i in range(len(corners)):
        temp_frame = img[bounds[0,i,1]:bounds[1,i,1],bounds[0,i,0]:bounds[1,i,0]]
        avg_values.append(get_avg_value(temp_frame))
    
    return avg_values
    
def merge_points(centers_old, centers_new, corners, avg_values, avg_values_corners, merge_threshold, distance_threshold):
    
    right_dist = []
    left_dist = []
    centers_new_update = centers_new.copy()
    avg_values_update = avg_values.copy()
    
    for i in range(len(corners)):
        right_dist.append(get_dist_line(centers_old[0],centers_old[1],corners[i]))
        left_dist.append(get_dist_line(centers_old[0],centers_old[2],corners[i]))

    l_x_dist = np.sqrt(np.sum(np.square(centers_new[0]-centers_new[1])))
    r_x_dist = np.sqrt(np.sum(np.square(centers_new[0]-centers_new[2])))
    
    if (l_x_dist < merge_threshold) and (np.min(left_dist) < distance_threshold):
        index = np.where(right_dist == np.min(left_dist))[0][0]
        centers_new_update[1] = corners[index]
        avg_values_update[1] = avg_values_corners[index]
    
    if (r_x_dist < merge_threshold) and (np.min(right_dist) < distance_threshold):
        index = np.where(right_dist == np.min(right_dist))[0][0]
        centers_new_update[2] = corners[index]
        avg_values_update[2] = avg_values_corners[index]
    
    return centers_new_update, avg_values_update

def get_bounds(centers, dist, im_size):
    
    bounds = np.array([centers - dist, centers + dist])
    
    bounds[bounds < 0] = 0 # set negative bounds to 0
    
    new_vals = bounds.copy()
    new_vals[new_vals > im_size[0]] = im_size[0]
    bounds[:,:,0] = new_vals[:,:,0] # set x bounds within x frame
    
    new_vals = bounds.copy()
    new_vals[new_vals > im_size[1]] = im_size[1]
    bounds[:,:,1] = new_vals[:,:,1] # set y bounds within y frame
    
    return bounds

def get_center_values(img, centers, dist, im_size):
    
    bounds = get_bounds(centers, dist, im_size)
    
    x_frame = img[bounds[0,0,1]:bounds[1,0,1],bounds[0,0,0]:bounds[1,0,0]]
    l_frame = img[bounds[0,1,1]:bounds[1,1,1],bounds[0,1,0]:bounds[1,1,0]]
    r_frame = img[bounds[0,2,1]:bounds[1,2,1],bounds[0,2,0]:bounds[1,2,0]]
    
    local_x_max = np.unravel_index(np.argmax(x_frame), x_frame.shape)
    local_l_max = np.unravel_index(np.argmax(l_frame), l_frame.shape)
    local_r_max = np.unravel_index(np.argmax(r_frame), r_frame.shape)
    
    # plt.imshow(x_frame,origin='lower')
    # plt.scatter(local_x_max[1], local_x_max[0],s=1,c='red')
    
    global_x_max = np.flip(np.ravel(local_x_max)) + [bounds[0,0,0], bounds[0,0,1]]
    global_l_max = np.flip(np.ravel(local_l_max)) + [bounds[0,1,0], bounds[0,1,1]]
    global_r_max = np.flip(np.ravel(local_r_max)) + [bounds[0,2,0], bounds[0,2,1]]
    
    centers_update = np.array([global_x_max, global_l_max, global_r_max])
    avg_values = np.array([get_avg_value(x_frame), get_avg_value(l_frame), get_avg_value(r_frame)])
    
    corners = get_corners(img)
    avg_values_corners = get_corner_values(img, corners, dist, im_size)
    new_centers, new_avg_vals = merge_points(centers, centers_update, corners, avg_values, avg_values_corners, 20, 10)
        
    return new_centers, new_avg_vals

def update_frame(img, centers, dist, im_size, threshold):
    
    new_centers, new_avg_vals = get_center_values(img, centers, dist, im_size)
    
    for i in range(3):
        if new_avg_vals[i] > threshold:
            centers[i] = new_centers[i]
    
    return centers, new_avg_vals

def main(input_array, centers_ini, dist, im_size, threshold, num_iter):
    centers = centers_ini.copy()
    centers_array = []
    avg_values_array = []
    
    for i in range(num_iter):
        img = input_array[i].copy()
        # norm=(img-np.min(img))/(np.max(img)-np.min(img))
        centers_update, avg_values_temp = update_frame(img, centers, dist, im_size, threshold)
        centers_array.append(centers_update)
        avg_values_array.append(avg_values_temp)
        centers = centers_update.copy()
    
    return np.array(centers_array), np.array(avg_values_array)

Finding the most likely points
1. Get distance to emission line between l and x or r and x. Remove points that are above a certain distance.
1. Remove points below a certain intensity
1. Get distance between individual points. Close points gets merged.
1. Check if there are more than 3 points. For n points over 3, perform n merges for the farthest points using point distance cost functions.
1. The emission x-point will always be given 100% certainty purely as a frame of reference to go by.

In [ ]:
x_0 = np.array([1.35,-1.05])
l_0 = np.array([1.0,-1.25])
r_0 = np.array([1.5,-1.25])

centers_0 = np.array([convert_center(radii,elevation,x_0),
                      convert_center(radii,elevation,l_0),
                      convert_center(radii,elevation,r_0)])

radius = 6
centers_array, avg_values_array = main(inverted, centers_0, radius, gray.shape, 0.05, len(inverted))

In [ ]:
plt.plot(avg_values_array)
plt.legend(['x','l','r'])
plt.show()

In [ ]:
idx = 199
x, y = zip(*centers_array[idx])
img = inverted[idx].copy()

fig, axs = plt.subplots()
circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), .005*radius, color='tab:red', fill=False)
circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), .005*radius, color='tab:red', fill=False)
circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), .005*radius, color='tab:red', fill=False)
line1x = [radii[0,x[0]],radii[0,x[1]]]
line1y = [elevation[0,y[0]],elevation[0,y[1]]]
line2x = [radii[0,x[0]],radii[0,x[2]]]
line2y = [elevation[0,y[0]],elevation[0,y[2]]]

plt.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='gray')
# axs.imshow(inverted[idx],origin='lower')
axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
axs.plot(line1x,line1y,color='tab:blue')
axs.plot(line2x,line2y,color='tab:blue')
axs.add_artist(circ_0)
axs.add_artist(circ_1)
axs.add_artist(circ_2)

gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x1 = radii[idx][corners[:,0,0]]
y1 = elevation[idx][corners[:,0,1]]
axs.scatter(x1,y1,color='cyan',s=5,marker='x',label='corners')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots()

def animate(idx):
    x, y = zip(*centers_array[idx])
    img = inverted[idx].copy()

    circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), .005*radius, color='tab:red', fill=False)
    circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), .005*radius, color='tab:red', fill=False)
    circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), .005*radius, color='tab:red', fill=False)
    
    line1x = [radii[0,x[0]],radii[0,x[1]]]
    line1y = [elevation[0,y[0]],elevation[0,y[1]]]
    line2x = [radii[0,x[0]],radii[0,x[2]]]
    line2y = [elevation[0,y[0]],elevation[0,y[2]]]

    axs.clear()
    
    plt.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='gray')
    axs.plot(line1x,line1y,color='tab:blue')
    axs.plot(line2x,line2y,color='tab:blue')
    axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
    axs.add_artist(circ_0)
    axs.add_artist(circ_1)
    axs.add_artist(circ_2)

writervideo = animation.FFMpegWriter(fps=15) 
ani = animation.FuncAnimation(fig, animate, frames=len(centers_array))
ani.save('animation.mp4', writer=writervideo)
plt.close()